In [49]:
from __future__ import print_function
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.datasets import mnist
from keras.models import Sequential    # Importing Sequential Model
from keras.layers.core import Dense,Dropout, Activation  #  Importing  Dense Layers,Dropouts and Activation functions
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils  
np.random.seed(1671) # for reproducibility -> Once you put the same seed you get same patterns of random numbers.

In [50]:
# network and training
NB_EPOCH = 30 # 30-> times the model is exposed to the training set.
BATCH_SIZE = 300
VERBOSE = 1
NB_CLASSES = 10 # number of outputs = number of digits
OPTIMIZER = Adam()
N_HIDDEN = 128 # Neurons
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3

In [52]:
# Data: shuffled and split between train and test sets

mnist_data = pd.read_csv("train.csv")

X_train = (mnist_data.iloc[:, 1:].values).astype('float32')

y_train_label = mnist_data['label'].values.astype('int32')

X_test = (pd.read_csv("test.csv").values).astype('float32')

# (X_train, y_train_label), (X_test, y_test_label) = mnist.load_data()



#X_train is 42000 rows of 28x28 values --> reshaped in 60000 x 784

RESHAPED = 784
X_train = X_train.reshape(42000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [53]:
# normalize -> Involve only rescaling to arrive at value relative to some size variables.

X_train /= 255 # Pixel values are 0 to 255 -> So we are normalizing training data by dividing it by 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

42000 train samples
28000 test samples


In [54]:
# convert class vectors to binary class matrices
Y_train_label = np_utils.to_categorical(y_train_label, NB_CLASSES) 


# np_utils.to_categorical Used to convert the array of labelled data to one Hot vector-> Binarization of category

In [55]:
# Final hidden layer  with 10 outputs
# final stage is softmax
model = Sequential() # Sequential Model.
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,))) # 1st Hidden Layer --> 128 neurons and input dimension ->784
model.add(Activation('relu')) # Activation function for 1st Hidden Layer
model.add(Dropout(DROPOUT))

model.add(Dense(N_HIDDEN))  # 2nd Hidden Layer --> 128 neurons
model.add(Activation('relu')) # Activation function for 2nd Hidden Layer
model.add(Dropout(DROPOUT))


model.add(Dense(NB_CLASSES)) # Final layer with 10 neurons == > no of outputs
model.add(Activation('softmax')) # Final layer activation will be 'softmax'

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 128)               100480    
_________________________________________________________________
activation_26 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_27 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 10)                1290      
__________

In [56]:
# Compiling a model in keras
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

In [57]:
# Training a model in keras

# Once the model is compiled it can be trained with the fit() function

history = model.fit(X_train, Y_train_label,
batch_size=BATCH_SIZE, epochs=NB_EPOCH,
verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 33600 samples, validate on 8400 samples
Epoch 1/30
33600/33600 [==============================] - 2s 63us/step - loss: 0.7847 - acc: 0.7601 - val_loss: 0.2776 - val_acc: 0.9202
Epoch 2/30
33600/33600 [==============================] - 2s 49us/step - loss: 0.3272 - acc: 0.9031 - val_loss: 0.1936 - val_acc: 0.9418
Epoch 3/30
33600/33600 [==============================] - 2s 49us/step - loss: 0.2458 - acc: 0.9260 - val_loss: 0.1555 - val_acc: 0.9549
Epoch 4/30
33600/33600 [==============================] - 2s 49us/step - loss: 0.2022 - acc: 0.9385 - val_loss: 0.1390 - val_acc: 0.9592
Epoch 5/30
33600/33600 [==============================] - 2s 49us/step - loss: 0.1734 - acc: 0.9483 - val_loss: 0.1222 - val_acc: 0.9632
Epoch 6/30
33600/33600 [==============================] - 2s 50us/step - loss: 0.1557 - acc: 0.9523 - val_loss: 0.1126 - val_acc: 0.9660
Epoch 7/30
33600/33600 [==============================] - 2s 51us/step - loss: 0.1383 - acc: 0.9579 - val_loss: 0.1101 - val_acc:

In [32]:
predictions = model.predict_classes(X_test, verbose=0)
submissions=pd.DataFrame({'ImageId':list(range(1,len(predictions) + 1)), "Label": predictions})
submissions.to_csv("Final_Prediction.csv", index=False, header=True)